<a href="https://colab.research.google.com/github/connorj4/serial-multibiometric-face-fingerprint/blob/master/cnn_fingerprint.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

gdrive_home_dir = '/content/drive/My Drive/'

Mounted at /content/drive


In [2]:
!ls '/content/drive/My Drive/dataset_fp/labelfinger.csv'
!ls '/content/drive/My Drive/dataset_fp/data/'


'/content/drive/My Drive/dataset_fp/labelfinger.csv'
10.jpg	18.jpg	25.jpg	32.jpg	3.jpg	47.jpg	54.jpg	61.jpg	69.jpg	76.jpg
11.jpg	19.jpg	26.jpg	33.jpg	40.jpg	48.jpg	55.jpg	62.jpg	6.jpg	77.jpg
12.jpg	1.jpg	27.jpg	34.jpg	41.jpg	49.jpg	56.jpg	63.jpg	70.jpg	78.jpg
13.jpg	20.jpg	28.jpg	35.jpg	42.jpg	4.jpg	57.jpg	64.jpg	71.jpg	79.jpg
14.jpg	21.jpg	29.jpg	36.jpg	43.jpg	50.jpg	58.jpg	65.jpg	72.jpg	7.jpg
15.jpg	22.jpg	2.jpg	37.jpg	44.jpg	51.jpg	59.jpg	66.jpg	73.jpg	80.jpg
16.jpg	23.jpg	30.jpg	38.jpg	45.jpg	52.jpg	5.jpg	67.jpg	74.jpg	8.jpg
17.jpg	24.jpg	31.jpg	39.jpg	46.jpg	53.jpg	60.jpg	68.jpg	75.jpg	9.jpg


In [0]:
from __future__ import print_function
import numpy as np

In [0]:
np.random.seed(1337)  # for reproducibility

In [5]:
import pandas as pd
from PIL import Image
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K

Using TensorFlow backend.


In [0]:
batch_size = 10
nb_classes = 10
nb_epoch = 2

In [0]:
# To make the data ready for CNN, pictures are named with indexes, like '1.jpg', '2.jpg', etc..
def dir_to_dataset(mypath, loc_train_labels=""):
    dataset = []
    
    gbr = pd.read_csv(loc_train_labels, sep="\t")

    #for file_count, file_name in enumerate( sorted(glob(glob_files),key=len) ):
    for i in range(1,81):
        image = Image.open(mypath + str(i)+'.jpg')
        img = Image.open(mypath + str(i)+'.jpg').convert('LA') #tograyscale
        '''
        pixels_list = list(img.getdata())
        for pixel in pixels_list:
          p = pixel
          dataset.append(p)
        '''
        #pixels_list = list(img.getdata())
        #width, height = img.size
        #pixels = [pixels[i * width:(i + 1) * width] for i in range(height)]
        
        pixels = [f[0] for f in list(img.getdata())] #<- original
        dataset.append(pixels)
    # outfile = glob_files+"out"
    # np.save(outfile, dataset)
    if len(loc_train_labels) > 0:
        df = pd.read_csv(loc_train_labels)
        return np.array(dataset), gbr["Class"].values
    else:
        return np.array(dataset)

In [13]:
if __name__ == '__main__':
    Data, y = dir_to_dataset("/content/drive/My Drive/dataset_fp/data/","/content/drive/My Drive/dataset_fp/labelfinger.csv")
    
    #Split the train set and validation set
    train_set_x = Data[:60]
    val_set_x = Data[10:]
    train_set_y = y[:60]
    val_set_y = y[10:]

    (X_train, y_train), (X_test, y_test) = (train_set_x,train_set_y),(val_set_x,val_set_y)

    # input image dimensions
    img_rows, img_cols = 640, 480
    # number of convolutional filters to use
    nb_filters = 32
    # size of pooling area for max pooling
    pool_size = (2, 2)
    # convolution kernel size
    kernel_size = (3, 3)
    
    

    # Checking if the backend is Theano or Tensorflow
    if K.image_dim_ordering() == 'th':
        X_train = X_train.reshape(X_train.shape[0], 1, img_rows, img_cols)
        X_test = X_test.reshape(X_test.shape[0], 1, img_rows, img_cols)
        input_shape = (1, img_rows, img_cols)
    else:
        X_train = X_train.reshape(X_train.shape[0], img_rows, img_cols, 1)
        X_test = X_test.reshape(X_test.shape[0], img_rows, img_cols, 1)
        input_shape = (img_rows, img_cols, 1) 

    X_train = X_train.astype('float32')
    X_test = X_test.astype('float32')
    X_train /= 255
    X_test /= 255
    print('X_train shape:', X_train.shape)
    print(X_train.shape[0], 'train samples')
    print(X_test.shape[0], 'test samples')

    # convert class vectors to binary class matrices
    Y_train = np_utils.to_categorical(y_train, nb_classes)
    Y_test = np_utils.to_categorical(y_test, nb_classes)
    
    model = Sequential()
    model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1],
                    border_mode='valid',
                    input_shape=input_shape))
    model.add(Activation('relu'))
    model.add(Convolution2D(nb_filters, kernel_size[0], kernel_size[1]))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=pool_size))
    model.add(Dropout(0.25))

    model.add(Flatten())
    #model.add(Dense(128))
    model.add(Dense(32))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(nb_classes))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy',
            optimizer='sgd',
            metrics=['accuracy'])

    model.fit(X_train, Y_train, batch_size=batch_size, nb_epoch=nb_epoch, verbose=1, validation_data=(X_test, Y_test))
    score = model.evaluate(X_test, Y_test, verbose=0)
    
    print(model.summary())
    
    print('Test score:', score[0])
    print('Test accuracy:', score[1])
    
    

X_train shape: (60, 640, 480, 1)
60 train samples
70 test samples


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:48: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), input_shape=(640, 480,..., padding="valid")`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:50: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3))`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:67: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 60 samples, validate on 70 samples
Epoch 1/2
60/60 [==============================] - 2s 38ms/step - loss: 4.7499 - acc: 0.0833 - val_loss: 2.3080 - val_acc: 0.1143
Epoch 2/2
60/60 [==============================] - 1s 19ms/step - loss: 2.3138 - acc: 0.0833 - val_loss: 2.2992 - val_acc: 0.1857
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 638, 478, 32)      320       
_________________________________________________________________
activation_13 (Activation)   (None, 638, 478, 32)      0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 636, 476, 32)      9248      
_________________________________________________________________
activation_14 (Activation)   (None, 636, 476, 32)      0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 